# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import rebound
import reboundx

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
rebx = reboundx.Extras(sim)
gr = rebx.load_force("gr_full")
gr.params["c"] = 10
gr.params["max_iterations"] = 100
rebx.add_force(gr)
sim.integrate(1e-300)

reb_res = jnp.array([[p.ax, p.ay, p.az] for p in sim.particles])
reb_res

Array([[ 0.05419503, -0.08164417,  0.00249506],
       [-0.04134904,  0.06280272, -0.00191058],
       [-0.02331262,  0.00210376,  0.07367954],
       [-0.02348517,  0.00205179, -0.07450105]], dtype=float64)

In [3]:
from jorbit.accelerations.gr import gr_full

sim = rebound.Simulation()
sim.add(m=1.0, x=-1.0, y=1.01, z=-0.05, vx=0.01, vy=0.02, vz=0.4)
sim.add(m=1.3, x=1.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.04, vx=-0.02, vy=-0.05, vz=-0.6)
sim.add(m=0.01, x=10.0, y=-2.01, z=0.4, vx=-0.02, vy=-0.05, vz=0.6)
sim.integrate(1e-300)
x0 = jnp.array([[p.x, p.y, p.z] for p in sim.particles])
v0 = jnp.array([[p.vx, p.vy, p.vz] for p in sim.particles])
gms = jnp.array([p.m for p in sim.particles])

jorb_res = gr_full(x=x0, v=v0, gms=gms, c2=100)

reb_res - jorb_res

Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.46944695e-18,  0.00000000e+00, -1.38777878e-17],
       [ 3.46944695e-18, -4.33680869e-19,  0.00000000e+00]],      dtype=float64)